In [1]:
# %%writefile xxx.py

import sys
import os 
module_path = os.path.abspath(os.path.join('..')) 
if module_path not in sys.path: 
    sys.path.append(module_path)

import numpy as np
import pandas as pd

import QUANTAXIS as QA

import tools.Sample_Tools as smpl
from base.JuUnits import excute_for_multidates

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker


In [49]:
min1 = smpl.get_data(['000001'], start='2020-01-03', end=None, gap=221, freq=QA.FREQUENCE.ONE_MIN)

In [50]:
min5 = smpl.get_data(['000001'], start='2020-01-03', end=None, gap=221, freq=QA.FREQUENCE.FIVE_MIN)


In [51]:
min1.data

,,open,high,low,close,volume,amount,type
datetime,code,,,,,,,
2020-01-03 09:31:00,000001,16.94,17.04,16.93,17.04,4038600.0,68552928.00,1min
2020-01-03 09:32:00,000001,17.04,17.10,17.04,17.08,2127200.0,36332396.00,1min
2020-01-03 09:33:00,000001,17.07,17.07,17.04,17.07,1604900.0,27383024.00,1min
2020-01-03 09:34:00,000001,17.07,17.15,17.06,17.13,2229600.0,38145524.00,1min
2020-01-03 09:35:00,000001,17.13,17.15,17.11,17.11,1671100.0,28628018.00,1min
...,...,...,...,...,...,...,...,...
2020-12-03 14:55:00,000001,19.48,19.49,19.46,19.48,406200.0,7906767.00,1min
2020-12-03 14:56:00,000001,19.48,19.48,19.46,19.47,137400.0,2675083.25,1min
2020-12-03 14:57:00,000001,19.47,19.48,19.46,19.48,145800.0,2838642.00,1min


In [46]:
min5.data

,,open,high,low,close,volume,amount,type
datetime,code,,,,,,,
2022-01-04 09:35:00,000001,16.48,16.50,16.25,16.35,9908000.0,162305520.0,5min
2022-01-04 09:40:00,000001,16.34,16.34,16.22,16.24,7251800.0,118065776.0,5min
2022-01-04 09:45:00,000001,16.26,16.28,16.21,16.28,6296100.0,102304512.0,5min
2022-01-04 09:50:00,000001,16.27,16.29,16.22,16.22,5007200.0,81343520.0,5min
2022-01-04 09:55:00,000001,16.20,16.25,16.18,16.23,5786300.0,93793536.0,5min
...,...,...,...,...,...,...,...,...
2022-12-01 14:40:00,000001,13.12,13.13,13.07,13.11,3582500.0,46936880.0,5min
2022-12-01 14:45:00,000001,13.11,13.14,13.11,13.11,3098300.0,40680328.0,5min
2022-12-01 14:50:00,000001,13.12,13.15,13.12,13.13,2520800.0,33116752.0,5min


In [47]:
min1.data.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 52818 entries, (Timestamp('2022-01-04 09:31:00'), '000001') to (Timestamp('2022-12-01 15:00:00'), '000001')
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   open    52818 non-null  float64
 1   high    52818 non-null  float64
 2   low     52818 non-null  float64
 3   close   52818 non-null  float64
 4   volume  52818 non-null  float64
 5   amount  52818 non-null  float64
 6   type    52818 non-null  object 
dtypes: float64(6), object(1)
memory usage: 8.2 MB


In [48]:
min5.data.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 10608 entries, (Timestamp('2022-01-04 09:35:00'), '000001') to (Timestamp('2022-12-01 15:00:00'), '000001')
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   open    10608 non-null  float64
 1   high    10608 non-null  float64
 2   low     10608 non-null  float64
 3   close   10608 non-null  float64
 4   volume  10608 non-null  float64
 5   amount  10608 non-null  float64
 6   type    10608 non-null  object 
dtypes: float64(6), object(1)
memory usage: 1.5 MB
